In [1]:
import pandas as pd
import requests
import json
import random
active_date = '2022-03-29'

In [2]:
price_history = pd.read_json('price_collection.json')
price_history.columns = ['Date', 'Ticker', 'Volume', 'Open', 'Low', 'High', 'Close']
price_history.sort_values('Date', ascending=True, inplace=True)
price_history = price_history[price_history['Date'] > '2023-01-01']
price_history = price_history.drop(columns=['Volume'])

In [3]:
iss_url = 'https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities.json?iss.meta=off&iss.only=marketdata&marketdata.columns=SECID,OPEN,LOW,HIGH,LAST'
response = requests.get(iss_url)
raw_data = json.loads(response.text)
#file_name = 'securities.json'
#with open(file_name, "r") as read_file:
#    raw_data = json.load(read_file)
#    read_file.close()

In [4]:
#SECID,OPEN,LOW,HIGH,LAST
for i in raw_data["marketdata"]["data"]:
    'Date', 'Ticker', 'Open', 'Low', 'High', 'Close'
    new_row = {'Date':active_date, 'Ticker':i[0], 'Open':i[1], 'Low':i[2], 'High':i[3], 'Close':i[4]} 
    price_history = price_history.append(new_row, ignore_index=True)

In [5]:
for selected_ticker in price_history.Ticker.unique(): 
    count_ema = price_history[price_history['Ticker'] == selected_ticker]
    count_ema = count_ema['Close'].to_frame()
    count_ema['Dayly_EMA'] = count_ema['Close'].ewm(span=20).mean()
    for indx in count_ema.index:
        price_history.at[indx, 'Dayly_EMA'] = count_ema.at[indx,'Dayly_EMA']

In [6]:
good_ticker = price_history[(price_history['Date'] == active_date) & (price_history['Open'] >= price_history['Dayly_EMA']) & (price_history['Dayly_EMA'] >= price_history['Low'])]
good_ticker['res'] = good_ticker['Close']/good_ticker['Dayly_EMA']

C:\Users\Jeda\AppData\Local\Temp/ipykernel_9836/1743291349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_ticker['res'] = good_ticker['Close']/good_ticker['Dayly_EMA']


In [7]:
kk_list = []
for j in range(0,1000000):
    res = 0
    for i in range(0,5):
        new_indx = random.choice(good_ticker['Ticker'].index)
        #print(good_ticker.at[new_indx, 'Ticker'])
        res += good_ticker.at[new_indx, 'res']
    #print(res)
    kk_list.append(res/5)

In [8]:
kk_avg = round((sum(kk_list)/len(kk_list)-1)*100,1)
plus = round(sum(True for i in kk_list if i > 1)/10000,1)
minkk = round((min(kk_list)-1)*100,1)
maxkk = round((max(kk_list)-1)*100,1)
print('За '+active_date+' условиям удовлетворяли '+str(len(good_ticker))+' бумаг')
print('Выполнен 1 млн моделирований слуйчайных комбинаций')
print('Средняя доходность '+str(kk_avg)+'%')
print('Вероятность получить доход '+str(plus)+'%')
print('Максимальная просадка '+str(minkk)+'%')
print('Максимальный доход '+str(maxkk)+'%')

За 2022-03-10 условиям удовлетворяли 36 бумаг
Выполнен 1 млн моделирований слуйчайных комбинаций
Средняя доходность 1.3%
Вероятность получить доход 99.9%
Максимальная просадка -0.4%
Максимальный доход 7.8%
